In [2]:
import torch
import torch.nn as nn

In [12]:
top_k = 2

In [13]:
logits = torch.randn((3, 10))

In [14]:
logits

tensor([[ 0.8006,  0.4391,  0.9963, -1.0577,  0.6655,  1.8054,  1.1058,  0.2714,
         -0.6118, -0.9285],
        [-0.6156,  0.1248, -0.2189, -1.5510, -0.3128,  0.2942,  0.0506,  1.3962,
         -0.2427, -0.4779],
        [-1.1814, -0.6649, -0.1693,  0.9384,  1.2404, -0.0670,  0.2987,  0.2910,
         -0.4496, -0.7796]])

In [15]:
topk_lrg = torch.topk(logits.T, k=top_k, dim=1, largest=True)

In [16]:
topk_lrg

torch.return_types.topk(
values=tensor([[ 0.8006, -0.6156],
        [ 0.4391,  0.1248],
        [ 0.9963, -0.1693],
        [ 0.9384, -1.0577],
        [ 1.2404,  0.6655],
        [ 1.8054,  0.2942],
        [ 1.1058,  0.2987],
        [ 1.3962,  0.2910],
        [-0.2427, -0.4496],
        [-0.4779, -0.7796]]),
indices=tensor([[0, 1],
        [0, 1],
        [0, 2],
        [2, 0],
        [2, 0],
        [0, 1],
        [0, 2],
        [1, 2],
        [1, 2],
        [1, 2]]))

In [8]:
import torch

softmax = torch.nn.Softmax()

a = torch.tensor([0.0000001, 0.0000001, 0.4, 0.6]) * 100
softmax(a).sum(), a / a.sum()

/tmp/ipykernel_25029/2123808535.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax(a).sum(), a / a.sum()


(tensor(1.), tensor([1.0000e-07, 1.0000e-07, 4.0000e-01, 6.0000e-01]))